In [ ]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from time import sleep
import re

In [ ]:
class Crawler_Gozine2:
    def __init__(self, url):
        self.url = url
        self.mainDriver = webdriver.Chrome()
        self.data = {'text':[]}
    
    def get_comment(self, index):
        element = self.mainDriver.find_element(By.XPATH, f'//*[@id="pnlOpinion"]/li[{index}]/div/div[2]')
        return element.text
    
    def run(self, max=10000):
        self.mainDriver.get(self.url)
        i = 1
        while True:
            try:
                if i > max:
                    break
                if i % 30 == 0:
                    self.mainDriver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
                comment = self.get_comment(i)
                self.data['text'].append(comment)
                i += 1
            except:
                break
        self.mainDriver.close()
        return pd.DataFrame(self.data)     
        

In [ ]:
url = 'https://article.gozine2.ir/view/46872/%D9%86%D8%B8%D8%B1%D8%B3%D9%86%D8%AC%DB%8C-%D8%A2%D8%B2%D9%85%D9%88%D9%86-%D9%87%D8%A7'
crawler = Crawler_Gozine2(url)
data = crawler.run()
data.to_csv("Gozine2_Azmoon.csv", index=False)

In [ ]:
class Crawler_MehroMah:
    def __init__(self):
        self.mainDriver = webdriver.Chrome()
        self.data = {'text' : []}
        
    def get_link(self, index):
        index = index*8+1
        element = self.mainDriver.find_element(By.XPATH, f'//*[@id="dle-content"]/div[{index}]/a')
        return element.get_attribute('href')

    def get_comments_number(self, index):
        index = index*8+3
        element = self.mainDriver.find_element(By.XPATH, f'//*[@id="dle-content"]/div[{index}]/div[4]/div/h3')
        number = int(re.findall(r'\d+', element.text)[0])
        return number
    
    def get_comment(self, index, driver):
        index += 3
        element = driver.find_element(By.XPATH, f"/html/body/div[3]/div[1]/div[1]/form/div/div[{index}]/div[1]/div/div[2]/div/div[2]")
        comment = element.text
        comment = comment.split('::')[0]
        return comment
    
    def handle_product_page(self, path, commentNumber):
        pageNumber = 1
        productDriver = webdriver.Chrome()
        while commentNumber > 0:
            link = f"https://home.mehromah.ir/products/page,1,{pageNumber},{path}"
            productDriver.get(link)
            for i in range(min(commentNumber, 15)):
                self.data['text'].append(self.get_comment(i, productDriver))
            commentNumber -= 15
            pageNumber += 1
        productDriver.close()
        
    
    def run(self, maxPage=53):
        for page in range(1, maxPage+1):
            try:
                self.mainDriver.get(f"https://home.mehromah.ir/products/page/{page}/")
                for i in range(15):
                    commentsNumber = self.get_comments_number(i)
                    if commentsNumber > 0:
                        link = self.get_link(i)
                        path = link.split('/products/')[-1]
                        self.handle_product_page(path, commentsNumber)
            except:
                pass
        data = pd.DataFrame(self.data)
        self.mainDriver.close()
        return data


In [ ]:
crawler = Crawler_MehroMah()
data = crawler.run()
data.to_csv('MehroMah_products_comments.csv', index=False)


In [ ]:

SEE_OTHER_CLASS_NAME = "jquerycomment-footer jquerycomment-refreshBtn jquerycomment-refreshBtn-1"
TIME = 2
data = None
class Crawler_Ghalamchi:
    def __init__(self):
        self.url = None
        self.mainDriver = webdriver.Chrome()
        self.data = {'text':[], 'comment':[]}
    
    def get_comment_text(self, index):
        element = self.mainDriver.find_element(By.XPATH, f'/html/body/div[1]/div/div[2]/comment/div[{index}]/div[3]/div[2]')                                                  
        return element.text
                                                            
    def get_reply_text(self, comment_index, reply_index):
        element = self.mainDriver.find_element(By.XPATH, f'/html/body/div[1]/div/div[2]/comment/div[{comment_index}]/div[{reply_index}]/div[2]/div[2]')
        return element.text 
    
    def get_reply_number(self, index):
        try:
            element = self.mainDriver.find_element(By.XPATH, f'/html/body/div[1]/div/div[2]/comment/div[{index}]/div[3]/div[3]/span[1]')
            number = re.findall('\d+', element.text)
            if len(number) == 0:
                return 0
            else:
                return int(number[0])
        except:
            return 0
        
    def see_other_comments(self, index):
        self.mainDriver.execute_script("window.scrollTo(0,document.body.scrollHeight)")
        sleep(2)
        element = self.mainDriver.find_element(By.XPATH, f'/html/body/div[1]/div/div[2]/comment/div[{index}]')
        if element.get_attribute('class') == SEE_OTHER_CLASS_NAME:
            action = ActionChains(self.mainDriver)
            action.move_to_element(element).click()
            action.perform() 
            sleep(TIME) 
        else:
            raise Exception()

    def see_other_replies(self, comment_index, reply_index):        
        element = self.mainDriver.find_element(By.XPATH, f'/html/body/div[1]/div/div[2]/comment/div[{comment_index}]/div[{reply_index}]')
        self.mainDriver.execute_script("arguments[0].scrollIntoView();",element)
        sleep(2)
        action = ActionChains(self.mainDriver)
        action.move_to_element(element).click()
        action.perform() 
        sleep(TIME)
        